<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Kaggle_%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9C_%EC%98%A4%EB%8B%B5_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np

In [27]:
# 랜덤 시드 고정
np.random.seed(2025)

# [문제 1 데이터] 서버 응답 시간 (로그 변환 필요)
# A서버: 분산 큼, B서버: 분산 작음
g1 = np.random.lognormal(mean=0, sigma=0.6, size=40)
g2 = np.random.lognormal(mean=0, sigma=0.4, size=35)
df_prob1 = pd.DataFrame({
    'response_time': np.concatenate([g1, g2]),
    'server': ['A']*40 + ['B']*35
})

df_prob1['response_time'] = np.log(df_prob1['response_time'])

#1
df_A = df_prob1[df_prob1['server'] == 'A']['response_time']
df_B = df_prob1[df_prob1['server'] == 'B']['response_time']

v_A = df_A.var()
v_B = df_B.var()


F = v_A/v_B
print(round(F, 4))

#2
len_A = len(df_A)
len_B = len(df_B)

res = ((len_A - 1)*v_A + (len_B - 1)*v_B ) / (len_A + len_B - 2)
print(round(res, 4))

#3
from scipy import stats
s,p = stats.ttest_ind(df_A, df_B, alternative='two-sided', equal_var = True)
print(round(p,4))


1.4546
0.2185
0.6312


In [33]:
# [문제 2 데이터] 다이어트 약물 효과
# 두 집단의 분산이 비슷하게 설정
np.random.seed(2025)

d1 = np.random.normal(loc=5, scale=2, size=25)
d2 = np.random.normal(loc=4, scale=1.8, size=30)
df_prob2 = pd.DataFrame({
    'weight_loss': np.concatenate([d1, d2]),
    'drug': ['X']*25 + ['Y']*30
})

#1
df_X = df_prob2[df_prob2['drug'] == 'X']['weight_loss']
df_Y = df_prob2[df_prob2['drug'] == 'Y']['weight_loss']
X_var = df_X.var()
Y_var = df_Y.var()
F = X_var/Y_var
print(round(F,3))

#2
X_len = len(df_X)
Y_len = len(df_Y)
res = ((X_len - 1)*X_var + (Y_len - 1)*Y_var) / (X_len + Y_len - 2)
print(round(res,3))

#3
from scipy import stats
s,p = stats.ttest_ind(df_X, df_Y, alternative='two-sided', equal_var = True)
print(round(p,3))

1.207
3.136
0.042


In [44]:
# [문제 3 데이터] 공장 인장 강도
# n수가 다르고 분산 차이가 조금 있는 경우
f1 = np.random.normal(loc=200, scale=10, size=15)
f2 = np.random.normal(loc=210, scale=15, size=20)
df_prob3 = pd.DataFrame({
    'strength': np.concatenate([f1, f2]),
    'machine': ['M1']*15 + ['M2']*20
})

#1
df_M1 = df_prob3[df_prob3['machine'] == 'M1']['strength']
df_M2 = df_prob3[df_prob3['machine'] == 'M2']['strength']
var_M1 = df_M1.var()
var_M2 = df_M2.var()
F = var_M2/var_M1
print(round(F,2))

#2
len_M1 = len(df_M1)
len_M2 = len(df_M2)
res = ((len_M1 - 1)*var_M1 + (len_M2 - 1)*var_M2) / (len_M1 + len_M2 - 2)
print(round(res,2))

#3
from scipy import stats
s,p = stats.ttest_ind(df_M1, df_M2, alternative='two-sided', equal_var = True)
print(round(p,2))

1.93
227.16
0.17


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

In [54]:
# 1. 시계열 데이터 (주문 내역)
df_order = pd.DataFrame({
    'order_id': range(1, 11),
    'date': pd.to_datetime(['2023-01-15', '2023-01-20', '2023-02-10', '2023-02-25', '2023-03-05',
                            '2023-04-10', '2023-05-01', '2023-06-15', '2023-07-20', '2023-08-30']),
    'amount': [100, 200, 150, 300, 120, 400, 250, 180, 350, 500]
})

#1
df_order['period'] = df_order['date'].dt.to_period('M')
grouped = df_order.groupby('period')['amount'].sum()
print(grouped)

#2
df_order['quarter'] = df_order['date'].dt.to_period('Q')
counts = df_order['quarter'].value_counts()
print(counts)

period
2023-01    300
2023-02    450
2023-03    120
2023-04    400
2023-05    250
2023-06    180
2023-07    350
2023-08    500
Freq: M, Name: amount, dtype: int64
quarter
2023Q1    5
2023Q2    3
2023Q3    2
Freq: Q-DEC, Name: count, dtype: int64


In [58]:
# 2. 범주형 데이터 (성적표)
df_score = pd.DataFrame({
    '반': ['A', 'A', 'B', 'B', 'A', 'B', 'C', 'C'],
    '과목': ['수학', '영어', '수학', '영어', '과학', '과학', '수학', '영어'],
    '점수': [90, 85, 88, 92, 95, 80, 70, np.nan] # 결측치 포함
})

#3
pivot = pd.pivot_table(df_score, index='반', columns='과목', values='점수', aggfunc='mean', fill_value=0)
print(pivot)

#4
pivot = pd.pivot_table(df_score, index = '과목', columns='반', values='점수', aggfunc='max')
print(pivot)

과목    과학    수학    영어
반                   
A   95.0  90.0  85.0
B   80.0  88.0  92.0
C    0.0  70.0   0.0
반      A     B     C
과목                  
과학  95.0  80.0   NaN
수학  90.0  88.0  70.0
영어  85.0  92.0   NaN


In [91]:
# 3. 수치형 데이터 (자동차 정보)
np.random.seed(42)
df_car = pd.DataFrame({
    'speed': np.random.randint(50, 150, 30),
    'weight': np.random.randint(1000, 3000, 30),
    'engine_size': np.random.randint(1000, 5000, 30),
    'distance': np.random.randint(10, 500, 30) # 종속변수
})

#1
from statsmodels.formula.api import ols
model = ols('distance~speed', data=df_car).fit()
conf = model.conf_int(alpha=0.05).loc['speed']
print(conf)

#2
model = ols('distance~speed+weight+engine_size', data=df_car).fit()
conf = model.conf_int(alpha=0.01)
print(conf.iloc[1:])

0   -1.091980
1    1.942734
Name: speed, dtype: float64
                    0         1
speed       -1.644166  2.635873
weight      -0.129381  0.099299
engine_size -0.042981  0.072887
